In [ ]:
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam
from keras.preprocessing.image import img_to_array
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from pyimagesearch.smallervggnet import SmallerVGGNet
import matplotlib.pyplot as plt
from imutils import paths
import numpy as np
import argparse
import random
import pickle
import cv2
import os

from keras.applications import MobileNetV2
from keras.models import load_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Activation
from keras.layers import Flatten
from keras.layers import Add
from keras.layers import Dense
from keras.layers import GlobalAveragePooling2D
from keras.models import Sequential

In [ ]:
import matplotlib
matplotlib.use("Agg")

In [ ]:
EPOCHS = 75
INIT_LR = 1e-3
BS = 32
IMAGE_DIMS = (224, 224, 3)

In [ ]:
# Load images
imagePaths = sorted(list(paths.list_images('./pose_dataset/train_set')))
random.seed(42)
random.shuffle(imagePaths)

In [ ]:
data = []
labels = []

# loop over the input images
for imagePath in imagePaths:
    # load the image, pre-process it, and store it in the data list
    image = cv2.imread(imagePath)
    image = cv2.resize(image, (IMAGE_DIMS[1], IMAGE_DIMS[0]))
    image = img_to_array(image)
    data.append(image)

    # extract set of class labels from the image path and update the
    # labels list
    l = label = imagePath.split(os.path.sep)[-2].split("_")
    labels.append(l)

In [ ]:
# scale the raw pixel intensities to the range [0, 1]
data = np.array(data, dtype="float") / 255.0
labels = np.array(labels)
print("[INFO] data matrix: {} images ({:.2f}MB)".format(
    len(imagePaths), data.nbytes / (1024 * 1000.0)))

# binarize the labels using scikit-learn's special multi-label
# binarizer implementation
print("[INFO] class labels:")
mlb = MultiLabelBinarizer()
labels = mlb.fit_transform(labels)

# loop over each of the possible class labels and show them
for (i, label) in enumerate(mlb.classes_):
    print("{}. {}".format(i + 1, label))

In [ ]:
def print_label_details(labels):

    print(len(labels))

    count_h = 0
    count_n = 0
    count_bending = 0
    count_couching = 0
    count_others = 0
    count_standing = 0
    for label in labels:
        if label[0] == 1:
            count_bending += 1
        if label[1] == 1:
            count_couching += 1
        if label[2] == 1:
            count_h += 1
        if label[3] == 1:
            count_n += 1
        if label[4] == 1:
            count_others += 1
        if label[5] == 1:
            count_standing += 1

    print("bending: {}\ncrouching: {}\n_h: {}\n_n: {} \nothers: {}\nstanding: {}".format(count_bending, count_couching, count_h, count_n, count_others, count_standing))

print_label_details(labels)

In [ ]:
mobilenetv2_model = MobileNetV2(input_shape=IMAGE_DIMS, weights=None, include_top=False)
mobilenetv2_model.summary()

In [ ]:
print(mobilenetv2_model.input)
print(mobilenetv2_model.layers[-1].output)
print(mlb.classes_)

# initialize the model using a sigmoid activation as the final layer
# in the network so we can perform multi-label classification

conv_model = Model(inputs=mobilenetv2_model.input,
                   outputs=mobilenetv2_model.layers[-1].output)

new_model = Sequential()
new_model.add(conv_model)
new_model.add(GlobalAveragePooling2D())
new_model.add(Dense(len(mlb.classes_), activation='sigmoid'))

new_model.summary()

In [ ]:
# partition the data into training and testing splits using 80% of
# the data for training and the remaining 20% for testing
(trainX, testX, trainY, testY) = train_test_split(data,
    labels, test_size=0, random_state=42)

# construct the image generator for data augmentation
aug = ImageDataGenerator(rotation_range=25, width_shift_range=0.1,
    height_shift_range=0.1, shear_range=0.2, zoom_range=0.2,
    horizontal_flip=True, fill_mode="nearest")

# initialize the optimizer (SGD is sufficient)
opt = Adam(lr=INIT_LR, decay=INIT_LR / EPOCHS)

# compile the model using binary cross-entropy rather than
# categorical cross-entropy -- this may seem counterintuitive for
# multi-label classification, but keep in mind that the goal here
# is to treat each output label as an independent Bernoulli
# distribution
new_model.compile(loss="binary_crossentropy", optimizer=opt,
    metrics=["accuracy"])

# train the network
print("[INFO] training network...")
H = new_model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
#     validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

In [ ]:

H = new_model.fit_generator(
    aug.flow(trainX, trainY, batch_size=BS),
#     validation_data=(testX, testY),
    steps_per_epoch=len(trainX) // BS,
    epochs=EPOCHS, verbose=1)

In [ ]:
# save the model to disk
print("[INFO] serializing network...")
new_model.save('pose_multi_labels_mobilenet_v2.h5')

In [ ]:
print("[INFO] serializing label binarizer...")
f = open('mlb.pickle', "wb")
f.write(pickle.dumps(mlb))
f.close()

In [ ]:
# plot the training loss and accuracy
plt.style.use("ggplot")
plt.figure()
N = EPOCHS
plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
# plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), H.history["acc"], label="train_acc")
# plt.plot(np.arange(0, N), H.history["val_acc"], label="val_acc")
plt.title("Training Loss and Accuracy")
plt.xlabel("Epoch #")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="upper left")
plt.savefig('plot.jpg')

In [ ]:
! git push

Username for 'https://github.com': 